# Imports  

In [ ]:
from gliderad2cp import process_adcp

# Section 1: calculate velocity shear

## 1.1 Begin ADCP Data Processing

Set paths to input files and specify options

In [ ]:
import pooch

server = "https://zenodo.org/records/10082274/files/"
mission = "sea057_M43"
data_source = pooch.create(
    path=pooch.os_cache("gliderad2cp"),
    base_url=server,
    registry={
        "sea057_M43.ad2cp.00000_1.nc": "sha256:775055eff485476af1cf79462e6ae400df04608bc1263434ea9a393e556d7b72",
        "sea057_M43.ad2cp.00000_2.nc": "sha256:71738df881c6a3d770b9c87f2d0607be979b1a75f2fa254c13720195b0cfa97e",
        "SEA057_M43.pqt": "2e72eda509d1c7027c9ed5ec70266b9bb161203e085c3de780aa9ee8ccb9cbce",
    },
)

In [ ]:
for filename in data_source.registry_files:
    data_source.fetch(filename)

In [ ]:
glider_pqt_path = data_source.path / 'SEA057_M43.pqt'
adcp_path = f"{data_source.path}/sea057_M43*.nc"

options = {
    'debug_plots' : True,
    'correctADCPHeading' : True,
    'ADCP_discardFirstBins' : 0, # First bin often very affected by glider, good to discard
    'ADCP_correlationThreshold': 70, # 70
    'ADCP_amplitudeThreshold' : 75, # 75
    'ADCP_velocityThreshold' : 0.8, # 0.6
    'correctXshear' : False,
    'correctYshear' : False,
    'correctZshear' : False,
    'correctZZshear' : False,
    'ADCP_regrid_correlation_threshold': 20,
    'plots_directory': 'plots',
    }


## 1.2 Load ADCP data and add necessary glider variables

In [ ]:
ADCP, data, options = process_adcp.load_adcp_glider_data(adcp_path, glider_pqt_path, options)

## 1.3 Start QC Calculate depth of each individual measurement


In [ ]:
ADCP = process_adcp.remapADCPdepth(ADCP, options)

## 1.4 Correct ADCP Heading

In [ ]:
ADCP = process_adcp.correct_heading(ADCP, data, options)

## 1.5 Perform soundspeed correction

In [ ]:
ADCP = process_adcp.soundspeed_correction(ADCP)

## 1.6 Outlier and poor correlation removal


Now we need to think. What do we discard, what do we keep, do we interpolate across missing bins?

In [ ]:
ADCP = process_adcp.remove_outliers(ADCP, options)

### 1.8 Correct shear bias

In [ ]:
ADCP = process_adcp.correct_shear(ADCP, options)

## 1.9 Remap 3-beam configuration onto isobars

In [ ]:
ADCP = process_adcp.regridADCPdata(ADCP, options)

## 1.10 Transform beams to XYZ and ENU velocities

In [ ]:
ADCP = process_adcp.calcXYZfrom3beam(ADCP, options)

In [ ]:
ADCP = process_adcp.calcENUfromXYZ(ADCP, options)

## 1.11 Grid shear

In [ ]:
xaxis, yaxis, taxis, days = process_adcp.grid_shear_data(data)
out = process_adcp.grid_data(ADCP, data, {}, xaxis, yaxis)
ds = process_adcp.make_dataset(out)

# 1.12 Write out data

**ds** is an xarray dataset of data gridded by depth bin and profile number, ready for processing

In [ ]:
ds.to_netcdf("gridded_shear.nc")

# Part 2: shear to velocity

After calculating velocity shear, there are several options for integrating and referencing this shear to obtain velocity profiles. This section demonstrates some of these options

# 2.1 Add extra variables for integrating shear

To calculate dive average current we require more variables, including estimates of the glider's movement through the water.

In [ ]:
import pandas as pd

extra_data = pd.read_parquet(glider_pqt_path)
extra_data.index = data.index
data["speed_vert"] = extra_data["speed_vert"]
data["speed_horz"] = extra_data["speed_horz"]
data["dead_reckoning"] = extra_data["dead_reckoning"]
data["nav_resource"] = extra_data["nav_state"]
data["dive_number"] = extra_data["dive_number"]

## 2.2. Calculate dive-averaged current

In [ ]:
data = process_adcp.get_DAC(ADCP, data)

## 2.3. Get surface drift

In [ ]:
dE,dN,dT = process_adcp.getSurfaceDrift(data)

## 2.4. Get bottom track data 
Bottom track data only calculated if the ADCP is bottom mounted

In [ ]:
ADCP = process_adcp.bottom_track(ADCP, adcp_path, options)

## 2.5. Reference ADCP data

Here we reference data to dive average current (DAC).

In [ ]:
out = process_adcp.reference_shear(ADCP, data, xaxis, yaxis)

## 2.6 Grid and plot data

In [ ]:
out = process_adcp.grid_data(ADCP, data, out, xaxis, yaxis)

In [ ]:
process_adcp.verify_depth_bias(out, yaxis)

### 2.7 Correct shear bias

In [ ]:
out = process_adcp.calc_bias(out, yaxis)

## 2.8 Create xarray DataSet from `out`

In [ ]:
ds = process_adcp.make_dataset(out)
ds.to_netcdf("gridded_velocity.nc")